In [ ]:
%run my_import.py

loaded_data.keys()
series_dict = {}
refresh_data = False

#per i plot nell'HTML
%matplotlib inline 


In [ ]:
# Sfondo nero e testi chiari per plot
mpl.rcParams.update({
    'figure.facecolor': 'black',
    'axes.facecolor': 'black',
    'axes.edgecolor': 'white',
    'axes.labelcolor': 'white',
    'xtick.color': 'white',
    'ytick.color': 'white',
    'text.color': 'white',
    'legend.facecolor': 'black',
    'legend.edgecolor': 'white',
    'legend.labelcolor': 'white',
    'savefig.facecolor': 'black',
    'savefig.edgecolor': 'black'
})

# Indice
- [WHITE NOISE](#white-noise)
- [OIL PRICE](#oil-price)
- [SOLAR ENERGY](#solar-energy)
- [MISSING DATA: SERIE UNIVARIATA DELLE SALES DI FAVORITA](#missing-data)
- [OUTLIERS DATA: SERIE UNIVARIATA DELLE SALES DI FAVORITA](#outliers-data)
- [F1 RACE DATA](#f1-race-data)
- [ELECTRICITY DEMAND](#electricity-demand)
- [FAVORITA SALES](#favorita-sales)

PER OGNI SERIE NEL NOTEBOOK
1) Descrizione serie storiche: dimensione, frequenza, descrizione generale.
2) Visualizzazione della serie
3) Analisi stazionarietà, test
4) Decomposizione serie: ciclicità, stagionalità, trend
5) Autocorrelazioni\
In presenza di covariate:
6) Descrizione covariate e relazioni col target 
7) Cross-correlation

### <a id="white-noise"></a> WHITE NOISE

In [ ]:
# 0) GENERAZIONE SERIE STORICA
n_years = 10
start_date = "2014-01-01"
series_name = "White Noise"
date_range = pd.date_range(start=start_date, periods=365*n_years, freq='D')
np.random.seed(42)
white_noise = np.random.normal(loc=0, scale=1, size=len(date_range))
series = pd.Series(white_noise, index=date_range)
series_dict[series_name] = series

In [ ]:
# 1) DESCRIZIONE SERIE STORICA
# -----------------------------------------------------
print("1. DESCRIZIONE DELLA SERIE STORICA")
print("-" * 50)

#### Informazioni generali sulla serie
print(f"Dimensione della serie: {series.shape[0]}")
print(f"Frequenza: {getattr(series.index, 'freq', 'Non specificata')}")
print("\nStatistiche descrittive:")
print(series.describe())
print("\nInformazioni aggiuntive sulla serie:")
print(series.info())


Serie univariata generata randomicamente (seed 42). Abbiamo 3650 osservazioni disponibili.

In [ ]:
# 2) VISUALIZZAZIONE SERIE STORICA
# -----------------------------------------------------
print("\n\n2. ANALISI STAZIONARIETÀ")
ts_plotly(series_dict, title=series_name)

In [ ]:
# 3) ANALISI STAZIONARIETÀ
# -----------------------------------------------------
print("\n\n3. ANALISI STAZIONARIETÀ")
print("-" * 50)
test_adf(series)
test_kpss(series)

Da qui notiamo la stazionarietà del White Noise. Sappiamo già a priori che la decomposizione non avrebbe senso, ma andiamo a vedere cosa succede.

In [ ]:
# 4) DECOMPOSIZIONE DELLA SERIE
# -----------------------------------------------------
print("\n\n4. DECOMPOSIZIONE DELLA SERIE")
print("-" * 50)
#non posso utilizzare la funzione che ho creato perchè ci sono 0 e valori negativi
try:
    decomposition = seasonal_decompose(series, model='additive', period=365)

    plt.figure(figsize=(14, 10))
    
    plt.subplot(411)
    plt.plot(series, label='Originale')
    plt.legend(loc='best')
    plt.title('Serie originale')
    
    plt.subplot(412)
    plt.plot(decomposition.trend, label='Trend')
    plt.legend(loc='best')
    plt.title('Trend')
    
    plt.subplot(413)
    plt.plot(decomposition.seasonal, label='Stagionalità')
    plt.legend(loc='best')
    plt.title('Stagionalità')
    
    plt.subplot(414)
    plt.plot(decomposition.resid, label='Residui')
    plt.legend(loc='best')
    plt.title('Residui')
    
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Impossibile eseguire la decomposizione stagionale: {e}")



In [ ]:
# 5) AUTOCORRELAZIONI
# -----------------------------------------------------
print("\n\n5. ANALISI DELLE CORRELAZIONI")
print("-" * 50)

# ACF - Autocorrelation Function
plot_acf(series, lags=40, alpha=0.05, title='Funzione di Autocorrelazione (ACF)')
plt.tight_layout()

# PACF - Partial Autocorrelation Function
plot_pacf(series, lags=40, alpha=0.05, title='Funzione di Autocorrelazione Parziale (PACF)')
plt.tight_layout()
plt.show()


Per un White Noise, ci aspettiamo che i valori di ACF e PACF siano vicini a zero per tutti i lag (tranne lag=0).
Se la maggior parte dei valori è all'interno delle bande di confidenza, questo conferma che la serie è effettivamente un white noise (nessuna autocorrelazione).

### <a id="oil-price"></a>OIL PRICE

In [ ]:
# 0) uploading data
oil = loaded_data['oil_prices']
oil.reset_index(inplace=True)
oil['date'] = oil['date'].dt.strftime('%Y-%m')
oil['date'] = pd.to_datetime(oil['date'])
oil.set_index('date', inplace=True)

Abbiamo caricato i dati e impostato l'indice temporale in formato Anno-mese

In [ ]:
# 1) DESCRIZIONE SERIE STORICA
# -----------------------------------------------------
print("1. DESCRIZIONE DELLA SERIE STORICA")
print("-" * 50)

#### Informazioni generali sulla serie
print(f"Dimensione della serie: {oil.shape[0]}")
print("\nStatistiche descrittive:")
print(oil.describe())
print("\nInformazioni aggiuntive sulla serie:")
print(oil.info())
# Verifica valori nulli
missing_values = oil.isnull().sum()
print("\nValori mancanti per colonna:")
print(missing_values)
if missing_values.sum() > 0:
    print("\nPercentuale di valori mancanti:")
    print(100 * missing_values / len(oil),"%")


**The Crude Oil price WTI (USD/Bbl)**\
In questa serie notiamo che non sono presenti dei missing values; i timesteps disponibili sono 506. Il 'Crude Oil WTI' è una qualità di petrolio greggio e uno dei principali benchmark per la determinazione dei prezzi del petrolio. Abbiamo rilevazioni mensili da marzo 1983 fino ad aprile 2025. Avremmo anche altre variabili come la variazione in valore assoluto e in percentuale.  

In [ ]:
# 2) VISUALIZZAZIONE SERIE STORICA
# -----------------------------------------------------
print("\n\n2. VISUALIZZAZIONE SERIE STORICA")
print("-" * 50)
ts_plotly({'Oil price': oil["price"]}, title='Oil price')

Utilizziamo plotly per la visualizzazione della serie, che ci permette anche di andare ad applicare metodi interattivi sul grafico, per capirne meglio l'andamento. I picchi positivi che notiamo sono a Giugno 2008 (appena prima della crisi finanziaria) e Maggio 2022. Notiamo anche tre shock negativi, il primo in corrispondenza della crisi finanziaria del 2008, il secondo nell'estate del 2014 e l'altro della crisi pandemica del 2020.

In [ ]:
# 2.1) Seasonal plot
# ===========================
plt.figure(figsize=(12, 6))
oil['year'] = oil.index.year
oil['month'] = oil.index.month
palette = sns.color_palette("viridis", n_colors=len(oil['year'].unique()))
for i, year in enumerate(oil['year'].unique()):
    yearly_data = oil[oil['year'] == year]
    monthly_price = yearly_data.groupby('month')['price'].mean()
    plt.plot(monthly_price, label=f'{year}', color=palette[i])

plt.title('Seasonal oil price plot')
plt.xlabel('Month')
plt.ylabel('Price')
plt.legend(title='Year', ncol=5, fontsize=5.5,loc="upper right")
plt.grid(True)
plt.show()

Il plot stagionale ci aiuta a capire meglio alcune dinamiche della serie storica in questione. Non sembra essere un fenomeno che ha dei picchi mensili o forte stagionalità. Possiamo apprezzare bene, però, l'andamento del prezzo greggio del petrolio negli anni, con le linee più scure che indicano anni 1983-2000 che sono più basse, quindi prezzo più basso. Le linee gialle sono le più recenti e segnalano un chiaro innalzamento del prezzo. Il picco più alto registrato è però verde, legato al giugno del 2008, come abbiamo visto in precedenza (a cui seguirà poi il grande crollo della crisi finanziaria). Notiamo infatti come le linee mediamente più alte sono le verdognole, cioè quelle dal 2008 al 2014 circa. Mentre le gialle, dal 2015 in poi, si assestano su valori, in media, leggermente più bassi. Questo grafico dà un altra prospettiva rispetto a ciò che avevamo visto nel plot precedente.

In [ ]:
# 3) ANALISI STAZIONARIETÀ
# -----------------------------------------------------
print("\n\n3. ANALISI STAZIONARIETÀ")
print("-" * 50)
test_adf(oil['price'])
test_kpss(oil['price'])

I test ci confermano che la serie non è stazionaria. Era prevedibile già vedendo il grafico soprastante.

In [ ]:
# 4) DECOMPOSIZIONE DELLA SERIE
# -----------------------------------------------------
print("\n\n4. DECOMPOSIZIONE DELLA SERIE")
print("-" * 50)
decomposition_add = seasonal_decompose(oil['price'], model='additive', period=12)
decomposition_mult = seasonal_decompose(oil['price'], 
                                            model='multiplicative', period=12)
plot_decomposition(decomposition_add, decomposition_mult)


Impostando period = 12, sto dicendo al modello che voglio depurare dell'effetto della stagionalità annuale. In altre parole, il modello si aspetta che i dati abbiano un ciclo stagionale che si ripete ogni 12 unità temporali. Ora, dal grafico possiamo aprezzare meglio il trend.\
Nel nostro caso siamo in presenza di volatily clustering, quindi all'aumentare del valore del prezzo, aumenta la varianza. In questo caso potrebbe essere più opportuno applicare una decomposizione moltiplicativa. Questo, in particolare, mi è suggerito dal grafico dei residui della decomposizione additiva. All'aumentare del tempo (abbiamo visto che aumenta anche il prezzo), aumenta la variabilità dei residui, rendendoli non-omeschedastici (gap tra <2008 e >2008), a differenza di quelli della decomposizione moltiplicativa che sembrano rispettare questa proprietà.\
Nella decomposizione moltiplicativa, la stagionalità e i residui vengono rappresentati come un fattore moltiplicativo, quindi è normale che oscillino intorno a 1. Dimostriamo ciò applicando il logaritmo naturale ai residui.

In [ ]:
plt.plot(np.log(decomposition_mult.resid))

Questa è una proprietà fondamentale delle trasformazioni logaritmiche: trasformano relazioni moltiplicative in relazioni additive.

In [ ]:
# 5) AUTOCORRELAZIONI
# -----------------------------------------------------
print("\n\n5. ANALISI DELLE CORRELAZIONI")
print("-" * 50)

# ACF - Autocorrelation Function
plot_acf(oil['price'].dropna(), lags=40, alpha=0.05, title='Funzione di Autocorrelazione (ACF)')
plt.tight_layout()

# PACF - Partial Autocorrelation Function
plot_pacf(oil['price'].dropna(), lags=40, alpha=0.05, title='Funzione di Autocorrelazione Parziale (PACF)')
plt.tight_layout()
plt.show()

L'ACF misura la correlazione tra una serie temporale e i suoi lag. In questo caso l'ACF va a zero molto lentamente, indicando che le osservazioni passate hanno un'influenza a lungo termine sulle osservazioni future.\
La PACF misura la correlazione tra una serie temporale e i suoi lag, eliminando l'influenza delle osservazioni intermedie. La PACF va a 0, dopo il lag 2. Questo potrebbe suggerire che la serie temporale può essere modellata efficacemente da un modello autoregressivo di ordine 2.

### <a id="solar-energy"></a> SOLAR ENERGY

In [ ]:
#Uploading data
SolarData = loaded_data['solar_power']
SolarData.set_index('DATE_TIME',inplace=True)

Carico i dati e imposto l'indice temporale

In [ ]:
# 1) DESCRIZIONE SERIE STORICA
# -----------------------------------------------------
print("1. DESCRIZIONE DELLA SERIE STORICA")
print("-" * 50)

#### Informazioni generali sulla serie
print(f"Dimensione della serie: {SolarData.shape[0]}")
print("Frequenza ogni 15 min -->>",SolarData.index[1:3])
print("\nInformazioni aggiuntive sulla serie:")
print(SolarData.info())
print("\nStatistiche descrittive:")
print(SolarData.describe())

Durante il periodo di osservazione dal 15 maggio 2020 al 17 giugno 2020, per un totale di 34 giorni, sono stati raccolti dati ogni 15 minuti, risultando in 3154 osservazioni temporali. Lo studio è stato condotto in India e noi abbiamo estratto i dati di un singolo invertitore (tutti molto simili fra loro) di energia della centrale solare 1. Nei dati raccolti non sono presenti valori mancanti.\
La variabile target è la potenza della corrente continua (DC_POWER) generata dall'invertitore, espressa in kilowatt (kW).\
Inizialmente questa serie verrà utilizzata come univariata. In una fase successiva si vorrà utilizzato alcuni regressori per migliorare il forecasting. Le variabili potenziali utilizzate come regressori includono la temperatura ambientale, la temperatura dei pannelli solari, l'irradiazione solare. \
È interessante notare che durante la notte la produzione di energia è completamente assente (0 kW), mentre durante il giorno varia tra 10k e 15k kW. Vediamo come i modelli performano con questa proprietà della serie. 

In [ ]:
# 2) VISUALIZZAZIONE SERIE STORICA
# -----------------------------------------------------
ts_plotly({'Solar Energy': SolarData["DC_POWER"]}, title='Direct Current')

Da questo grafico possiamo apprezzare bene principale caratteristica della serie. Durante la notte i kW vanno a 0, mentre durante il giorno, in presenza della luce del sole, raccolgono e distribuiscono energia. Grande ciclicità.

In [ ]:
# 3) ANALISI STAZIONARIETÀ
# -----------------------------------------------------
print("\n\n3. ANALISI STAZIONARIETÀ")
print("-" * 50)
test_adf(SolarData["DC_POWER"])
test_kpss(SolarData["DC_POWER"])

Questa serie è stazionaria nel senso che media e varianza costanti nel tempo e se le autocorrelazioni non dipendono dal tempo. Una serie può avere un forte pattern ciclico giornaliero ma essere comunque considerata "stazionaria" se queste fluttuazioni si ripetono in modo costante e prevedibile. Il pattern up-down potrebbe essere interpretato come una proprietà stabile della serie. I test ADF e KPSS sono progettati principalmente per rilevare trend

In [ ]:
# 4) DECOMPOSIZIONE DELLA SERIE
# -----------------------------------------------------
print("\n\n3. DECOMPOSIZIONE DELLA SERIE")
print("-" * 50)

decomposition_add = seasonal_decompose(SolarData['DC_POWER'], model='additive', period=96)
decomposition_mult = seasonal_decompose(SolarData['DC_POWER'].replace(0, 0.1), 
                                            model='multiplicative', period=96)
plot_decomposition(decomposition_add, decomposition_mult)



**Grafici del trend**: queste variazioni rappresentano l'andamento medio della potenza, escludendo l'effetto del ciclo giornaliero. Un andamento complessivo relativamente stabile intorno ai 2000-3500 kW, esclusi due grandi picchi (microperiodi di pioggia VS microperiodi di sole).\
**Grafici della stagionalità**: ciclo perfettamente ripetitivo che mostra la variazione attesa durante le 24 ore. Quando la stagionalità è inferiore a 0 (o 1 nel caso della moltiplicativa), riduce il valore previsto dal trend. Quando la stagionalità è superiore a 0 (o 1), amplifica il valore previsto dal trend\
**Grafici dei residui**: Il modello additivo sembra non stia riuscendo a cogliere qualcosa. \
Per il modello moltiplicativo ci sono dei picchi anomali superiore a 15, che potrebbero rappresentare un evento straordinario.\
Per consentire la decomposizione moltiplicativa ho sostituito tutti gli 0 con 0.1.


In [ ]:
# 5) AUTOCORRELAZIONI
# -----------------------------------------------------
print("\n\n4. ANALISI DELLE CORRELAZIONI")
print("-" * 50)

# ACF - Autocorrelation Function
plot_acf(SolarData['DC_POWER'].dropna(), lags=200, alpha=0.05, title='Funzione di Autocorrelazione (ACF)')
plt.tight_layout()

# PACF - Partial Autocorrelation Function
plot_pacf(SolarData['DC_POWER'].dropna(), lags=25, alpha=0.05, title='Funzione di Autocorrelazione Parziale (PACF)')
plt.tight_layout()
plt.show()


Il grafico dell'ACF mostra un chiaro pattern ciclico con una forte correlazione periodica. Questo è caratteristico di dati con una forte componente stagionale, legata al ciclo giornaliero del sole
- Correlazioni positive elevate vicino a lag 0 (come atteso) che decrescono gradualmente 
- Un andamento sinusoidale che si ripete, indicando una forte stagionalità 
- Zone di correlazione negativa che alternano quelle positive; questo pattern si ripete approssimativamente ogni 96 lag, cioè 24 ore

La PACF mostra una forte autocorrelazione parziale al lag 1 2 e lievemente anche al 3.\
Questi pattern descrivono chiaramente stagionalità giornaliera: la serie temporale mostra una forte componente ciclica giornaliera, legata al ciclo solare, che determina la produzione di energia fotovoltaica.

##### ANALISI DELLE FEATURE 

In [ ]:
#aggiungo features
df = SolarData.copy()
df_reset = df.reset_index()
df_reset['DATE_TIME'] = pd.to_datetime(df_reset['DATE_TIME'])
df_reset['giorno_settimana'] = df_reset['DATE_TIME'].dt.dayofweek + 1
df_reset['ora_del_giorno'] = df_reset['DATE_TIME'].dt.hour + 1
df = df_reset.set_index('DATE_TIME')

#plot della DC_POWER vs ora del giorno
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='ora_del_giorno', y='DC_POWER')
plt.title('DC Power vs Hour of the Day')
plt.xlabel('Hour of the Day')
plt.ylabel('DC Power')
plt.xticks(rotation=45)
plt.grid()
plt.show()

Questo boxplot riassume quanto detto finora. Chiara dipendenza (non lineare) della corrente diretta, dall'ora del giorno. Dalle 20 alle 6 è la produzione è esattamente 0.\
-->> #df.groupby('ora_del_giorno')['DC_POWER'].sum().

In [ ]:
# 6) ANALISI CON COVARIATE
# -----------------------------------------------------
# 6.1) DISTRIBUZIONE DELLE VARIABILI
# -----------------------------------------------------
print("\n\n6.1 DISTRIBUZIONE DELLE VARIABILI")
print("-" * 50)

numeric_vars = ['DC_POWER', 'AC_POWER', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION']

# Plot delle distribuzioni - solo istogrammi, senza boxplot
fig, axes = plt.subplots(len(numeric_vars), 1, figsize=(16, 3*len(numeric_vars)))
fig.suptitle('Distribuzione delle Variabili', fontsize=16, y=0.99)

for i, var in enumerate(numeric_vars):
    # Histogram
    sns.histplot(df[var], kde=True, ax=axes[i])
    axes[i].set_title(f'Distribuzione di {var}')
    axes[i].set_xlabel(var)
    axes[i].set_ylabel('Frequenza')

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

# -----------------------------------------------------
# 6.2) MATRICE DI CORRELAZIONE
# -----------------------------------------------------
print("\n\n6.2 MATRICE DI CORRELAZIONE")
print("-" * 50)

# Calcoliamo la matrice di correlazione per le variabili numeriche
corr_matrix = df[numeric_vars].corr()

# Visualizziamo la matrice di correlazione con annotazioni
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(corr_matrix, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0,
            annot=True, fmt=".2f", square=True, linewidths=.5)

plt.title('Matrice di Correlazione', fontsize=16)
plt.tight_layout()
plt.show()

print("Correlazioni con DC_POWER:")
for var in numeric_vars:
    if var != 'DC_POWER':
        corr, p_value = pearsonr(df['DC_POWER'], df[var])
        print(f"- {var}: {corr:.4f} (p-value: {p_value:.4g})")
# -----------------------------------------------------
# 6.3) RELAZIONI TRA VARIABILI 
# -----------------------------------------------------
print("\n\n6.3 RELAZIONI TRA VARIABILI")
print("-" * 50)

# Pairplot per le variabili numeriche
sns.pairplot(df[numeric_vars], height=2.5, corner=True)
plt.suptitle('Pairplot delle Variabili Numeriche', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

# -----------------------------------------------------
# 6.4) RELAZIONI CON IL TARGET (DC_POWER)
# -----------------------------------------------------
print("\n\n6.4 RELAZIONI CON IL TARGET (DC_POWER)")
print("-" * 50)

# Definisci le variabili predittori
predictors = ['AC_POWER', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION']

# Crea i scatter plot con linea di regressione
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, var in enumerate(predictors):
    sns.regplot(x=df[var], y=df['DC_POWER'], ax=axes[i], scatter_kws={'alpha':0.3}, line_kws={'color':'red'})
    axes[i].set_title(f'DC_POWER vs {var}')
    axes[i].set_xlabel(var)
    axes[i].set_ylabel('DC_POWER')

plt.tight_layout()
plt.show()

# -----------------------------------------------------
# 6.5) ANALISI TEMPORALE
# -----------------------------------------------------
print("\n\n6.5 ANALISI TEMPORALE")
print("-" * 50)

fig, ax1 = plt.subplots(figsize=(16, 8))
        
ax1.set_xlabel('Data')
ax1.set_ylabel('DC_POWER', color='tab:blue')
ax1.plot(df.index, df['DC_POWER'], color='tab:blue', label='DC_POWER')
ax1.tick_params(axis='y', labelcolor='tab:blue')
        
ax2 = ax1.twinx()  # crea un secondo asse y condividendo l'asse x
ax2.set_ylabel('AMBIENT TEMPERATURE', color='tab:orange')  
ax2.plot(df.index, df['AMBIENT_TEMPERATURE'], color='tab:orange', label='IRRADIATION')
ax2.tick_params(axis='y', labelcolor='tab:orange')
        
fig.tight_layout()
plt.title('DC_POWER e AMBIENT_TEMPERATURE nel Tempo')
        
# Aggiungi legenda
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')
plt.show()
        
# -----------------------------------------------------
# 6.6) HEATMAP PER ORA E GIORNO
# -----------------------------------------------------
print("\n\n6.6 HEATMAP PER ORA E GIORNO")
print("-" * 50)

# Creazione di una pivot table per la heatmap
pivot_data = df.pivot_table(index='ora_del_giorno', 
                            columns='giorno_settimana', 
                            values='DC_POWER', 
                            aggfunc='mean')

plt.figure(figsize=(12, 10))
sns.heatmap(pivot_data, cmap='YlOrRd', annot=True, fmt=".1f", linewidths=.5)
plt.title('DC_POWER Media per Ora e Giorno')
plt.xlabel('Giorno della Settimana')
plt.ylabel('Ora del Giorno')
plt.tight_layout()
plt.show()

I primi grafici delle distribuzioni delle variabili non ci aiutano molto perchè la maggior parte dei valori è 0 per quanto abbiamo detto finora. \
I grafici successivi dimostrano grande correlazione positiva fra tutte le variabili che abbiamo (la più bassa, 0.72, fra temperatura ambiente e corrente diretta, viene mostrata nel grafico 6.5).\
Interessante l'heatmap nella sezione 6.6. Le celle in rosso scuro sono quelle con più produzione di energia e ovviamente sono quelle ore che vanno dalle 9 di mattia alle 16, indicativamente. E' strano notare come la domenica i pannelli rilevino meno energia, forse a causa di chiusure della centrale solare? O manutenzioni/pulizie settimanali programmate? La grande differenza è fra le 13 e le 15, in cui la domenica si ricava circa la metà di kW. Per le altre fasce orarie siamo in linea con gli altri giorni.

In [ ]:
# 7) CROSS-CORRELATION
s1= (df['DC_POWER'] - df['DC_POWER'].mean()) / df['DC_POWER'].std()
s2 = (df['AMBIENT_TEMPERATURE'] - df['AMBIENT_TEMPERATURE'].mean()) / df['AMBIENT_TEMPERATURE'].std()

cross_corr_norm = np.correlate(s1, s2, mode='full')
lags = np.arange(-len(s1)+1, len(s2))

plt.figure(figsize=(10, 4))
plt.plot(lags, cross_corr_norm)
plt.axhline(0, color='gray', linestyle='--')
plt.title("Cross-correlation between DC_POWER e la temperature ambiente")
plt.xlabel("Lag")
plt.ylabel("Correlation")
plt.grid(True)
plt.tight_layout()
plt.show()

L'obiettivo della cross correlation è capire se una serie anticipa o ritarda un'altra; oppure se variano insieme.\
La funzione np.correlate() di NumPy calcola la correlazione tra due sequenze di dati. in np.correlate, la seconda serie viene "fissata" e si fa "scorrere" la prima sopra di essa. \
np.correlate() calcola la convoluzione discreta di due sequenze, che può essere utilizzata per determinare la similarità tra le due serie di dati in vari punti di lag. \
Il mode può essere 'valid': Restituisce solo i punti in cui le sequenze si sovrappongono completamente.\
'same': Restituisce un output della stessa lunghezza della sequenza più lunga.\
'full': Restituisce la convoluzione completa, includendo i punti in cui le sequenze si sovrappongono parzialmente.

INTERPRETAZIONE CROSS-CORRELATION:
1. Un picco di correlazione positivo a lag positivo indica che la prima serie anticipata 'prevede' la seconda.
2. Un picco di correlazione positivo a lag negativo indica che la seconda serie anticipata 'prevede' la prima.
3. Un picco di correlazione negativo indica una relazione inversa tra le serie a quel particolare lag.
4. La larghezza del picco di correlazione indica quanto è 'persistente' la relazione nel tempo.
5. Serie non correlate mostrano pattern casuali senza picchi definiti nella cross-correlazione.

Nel nostro caso, i picchi più alti delle correlazioni si trovano intorno al lag 0, indicando che che la relazione più forte tra temperatura e produzione avviene con poco o nessun ritardo temporale. Guardando bene il picco positivo di correlazione si trova con un lago negativo (forse lag -1), indicando che la temperatura ambiente potrebbe anticipare/prevedere la DC POWER.\
Il pattern oscillatorio che si estende in entrambe le direzioni (lag positivi e negativi) suggerisce una ciclicità nei dati.\
La diminuzione dell'ampiezza delle oscillazioni man mano che ci si allontana dal centro (lag 0) suggerisce che la correlazione diventa più debole con l'aumentare del ritardo temporale tra i due segnali.\
Questo tipo di pattern è tipico per dati solari, dove sia la temperatura che la produzione di energia seguono cicli giornalieri

### <a id="missing-data"></a>**missing data**: serie univariata delle sales di Favorita

In [ ]:
#uploading and distorcing data
missing = loaded_data['favorita_train']
missing.set_index("date", inplace=True)
missing = missing[['sales','family']]
#sommo tutte le vendite di un singolo store per ogni giorno
missing = missing.groupby(['date']).sum().reset_index()
missing.drop(columns=["family"], inplace=True)
missing = missing.rename(columns={"sales": "total_daily_sales"})
missing.set_index("date", inplace=True)

np.random.seed(24)
nan_indices = np.random.choice(missing.index, size=int(len(missing) * 0.2), replace=False)
missdata = missing.copy()
missdata.loc[nan_indices, 'total_daily_sales'] = np.nan

Abbiamo preso il dataset favorita (che ci servirà anche dopo per analisi panel) e sommato tutte le vendite di un singolo store per ogni giorno. Tutto ciò per avere un unica serie storica su cui testare questi due esperimenti.\
In questo caso abbiamo introdotto un 20% di NaN randomico con set_seed(24). L'obiettivo sperimentale è capire come TFT e Chronos gestiscono e reagiscono in presenza di molti valori mancanti.

In [ ]:
# 1) DESCRIZIONE SERIE STORICA
# -----------------------------------------------------
print("1. DESCRIZIONE DELLA SERIE STORICA")
print("-" * 50)

#### Informazioni generali sulla serie
print(f"Dimensione della serie: {missdata.shape[0]}")
print(f"Frequenza: giornaliera")
print("\nStatistiche descrittive:")
print(missdata.describe().round(1))
print("\nInformazioni aggiuntive sulla serie:")
print(missdata.info())
# Verifica valori nulli
missing_values = missdata.isnull().sum()
print("\nValori mancanti per colonna:")
print(missing_values)
if missing_values.sum() > 0:
    print("\nPercentuale di valori mancanti:")
    print(f"{100 * missing_values / len(missdata)}")


1348 osservazioni disponibili NON mancanti.

In [ ]:
# 2) VISUALIZZAZIONE SERIE STORICA
# -----------------------------------------------------
print("\n\n3. VISUALIZZAZIONE SERIE STORICA")
print("-" * 50)
ts_plotly({"sales": missdata['total_daily_sales']},
          title="Sales for store 1 with 20% of NaN")

Notiamo subito come ogni primo dell'anno questo store Favorita è chiuso.

In [ ]:
# 3) ANALISI STAZIONARIETÀ
# -----------------------------------------------------
print("\n\n3. ANALISI STAZIONARIETÀ")
print("-" * 50)
test_adf(missdata['total_daily_sales'])
test_kpss(missdata['total_daily_sales'])

Entrambi i test ci confermano che la serie non è stazionaria

La funzione per la decomposizione non accetta i NaN, quindi prendiamo il dataset iniziale senza NaN per capire cosa abbiamo

In [ ]:
# 4) DECOMPOSIZIONE DELLA SERIE, period = 365
# -----------------------------------------------------
print("\n\n4. DECOMPOSIZIONE DELLA SERIE, period = 365")
print("-" * 50)
decomposition_add = seasonal_decompose(missing['total_daily_sales'], model='additive', period=365)
decomposition_mult = seasonal_decompose(missing['total_daily_sales'].replace(0, 0.1), 
                                            model='multiplicative', period=365)
plot_decomposition(decomposition_add, decomposition_mult)

In [ ]:
# 4) DECOMPOSIZIONE DELLA SERIE, period = 7
# -----------------------------------------------------
print("\n\n4. DECOMPOSIZIONE DELLA SERIE, period = 7")
print("-" * 50)
decomposition_add = seasonal_decompose(missing['total_daily_sales'], model='additive', period=7)
decomposition_mult = seasonal_decompose(missing['total_daily_sales'].replace(0, 0.1), 
                                            model='multiplicative', period=7)
plot_decomposition(decomposition_add, decomposition_mult)

In questo caso, fra decomposizione additiva e moltiplicativa non c'è molta differenza.\
Con la decomposizione annuale capisco meglio il trend di lungo termine.\
Usando 7 (settimanale) come periodo per la decomposizione stagionale, catturo picchi durante il weekend e relazioni fra stessi giorni di settimane diverse \
Utili entrambe.

In [ ]:
# 5) AUTOCORRELAZIONI
# -----------------------------------------------------
print("\n\n5. ANALISI DELLE CORRELAZIONI")
print("-" * 50)

# ACF - Autocorrelation Function
plot_acf(missing['total_daily_sales'], lags=50, alpha=0.05, title='Funzione di Autocorrelazione (ACF)')
plt.tight_layout()

# PACF - Partial Autocorrelation Function
plot_pacf(missing['total_daily_sales'], lags=50, alpha=0.05, title='Funzione di Autocorrelazione Parziale (PACF)')
plt.tight_layout()
plt.show()


**ACF:** Si notano picchi significativi che si ripetono ciclicamente, ai lag 6/7/8, 13/14/15 etc....\
Questo pattern ciclico suggerisce una forte stagionalità nelle vendite, probabilmente settimanale.\
**PACF:** Picchi significativi ai lag 7, 14 e in parte al lag 21. I valori tendono a diminuire progressivamente all'aumentare dei lag. Dopo il lag 28 circa, la maggior parte dei valori è vicina allo zero. \
Le vendite di un giorno sono fortemente influenzate da quelle del giorno precedente (alto valore PACF al lag 1)

### <a id="outliers-data"></a>**outliers data**: serie univariata delle sales di Favorita

E' la stessa di prima: senza valori mancanti, abbiamo aggiunto 11 outlier.

In [ ]:
# 0) Uploading and distorcing data
outl = missing.copy()
def create_outliers():
    # Creo qualche outlier in maniera randomica
    np.random.seed(22)
    # Genero 7 valori random tra 2,000,000 e 4,000,000
    outliers_positive = np.random.randint(2000000, 4000000, 7)
    # Genero 4 valori random tra -2,000,000 e -4,000,000
    outliers_negative = np.random.randint(-3000000, -1000000, 4)
    outliers = np.concatenate((outliers_positive, outliers_negative))
    return outliers

outliers = create_outliers()
np.random.seed(22)
random_indices = np.random.choice(outl.size, 11, replace=False)
flat_df = outl.values.flatten()
flat_df[random_indices] = outliers
out = pd.DataFrame(flat_df.reshape(outl.shape), columns=outl.columns)

In [ ]:
# 1) DESCRIZIONE SERIE STORICA
# -----------------------------------------------------
print("1. DESCRIZIONE DELLA SERIE STORICA")
print("-" * 50)

#### Informazioni generali sulla serie
print(f"Dimensione della serie: {out.shape[0]}")
print(f"Frequenza: giornaliera")
print("\nStatistiche descrittive:")
print(out.describe().round(1))
print("\nInformazioni aggiuntive sulla serie:")
print(out.info())
# Verifica valori nulli
missing_values = out.isnull().sum()
print("\nValori mancanti per colonna:")
print(missing_values)
if missing_values.sum() > 0:
    print("\nPercentuale di valori mancanti:")
    print(f"{100 * missing_values / len(out)}")


In [ ]:
# 2) VISUALIZZAZIONE SERIE STORICA
# -----------------------------------------------------
print("\n\n3. VISUALIZZAZIONE SERIE STORICA")
print("-" * 50)
ts_plotly({"sales": out['total_daily_sales']},
          title="Sales for store 1 with 20% of NaN")

In [ ]:
# 3) ANALISI STAZIONARIETÀ
# -----------------------------------------------------
print("\n\n3. ANALISI STAZIONARIETÀ")
print("-" * 50)
test_adf(out['total_daily_sales'])
test_kpss(out['total_daily_sales'])

4) Non è possibile applicare le funzioni di decomposizione con valori negativi o 0

In [ ]:
# 5) AUTOCORRELAZIONI
# -----------------------------------------------------
print("\n\n5. ANALISI DELLE CORRELAZIONI")
print("-" * 50)

# ACF - Autocorrelation Function
plot_acf(out['total_daily_sales'], lags=50, alpha=0.05, title='Funzione di Autocorrelazione (ACF)')
plt.tight_layout()

# PACF - Partial Autocorrelation Function
plot_pacf(out['total_daily_sales'], lags=50, alpha=0.05, title='Funzione di Autocorrelazione Parziale (PACF)')
plt.tight_layout()
plt.show()


Stessa intepretazione di prima

### <a id="f1-race-data"></a>F1 RACE DATA

In [ ]:
# 0) Uploading data
f1 = loaded_data['f1']
f1.set_index("Date",inplace=True)

Serie storica IOT, non equidistribuita

In [ ]:
# 1) DESCRIZIONE SERIE STORICA
# -----------------------------------------------------
print("1. DESCRIZIONE DELLA SERIE STORICA")
print("-" * 50)

#### Informazioni generali sulla serie
print(f"Dimensione della serie: {f1.shape[0]}")
print("Frequenza ogni 120/130 millisecondi -->>",f1.index[1:3])
print("\nStatistiche descrittive:")
print(f1.describe())
print("\nInformazioni aggiuntive sulla serie:")
print(f1.info())

Abbiamo i dati per 20 piloti di F1, durante la gara di Silverstone 2023; dati rilevati dai sensori delle proprie autovetture, ogni 120/130 millisecondi (leggermente variabile). Le variabili sono:\
'DistanceToDriverAhead': distanza dal pilota che si ha davanti\
'RPM': ripetizione per minuti del motore\
'Speed': velocità in km/h\
'nGear' : marcia\
'Throttle': Percentuale di acceleratore utilizzato (%)\
'DRS': dummy per capire se il DRS è attivo o meno\
'LapNumber' : Giri effettuati nel GP

In [ ]:
# 2) VISUALIZZAZIONE SERIE STORICA
# -----------------------------------------------------
print("2. VISUALIZZAZIONE SERIE STORICA")
print("-" * 50)

series_dict = {
    'LEC': f1[f1["Driver"]=="LEC"]["RPM"],
    'NOR': f1[f1["Driver"]=="NOR"]["RPM"]}
ts_plotly(series_dict, title="Engine RPM")

Variabile RPM per due diversi piloti, Leclerc e Norris

In [ ]:
# 3) ANALISI STAZIONARIETÀ
# -----------------------------------------------------
#prendiamo un pilota a caso per vedere la stazionarietà
print("\n\n3. ANALISI STAZIONARIETÀ")
print("-" * 50)
test_adf(f1[f1["Driver"]=="LEC"]["RPM"])
test_kpss(f1[f1["Driver"]=="LEC"]["RPM"])

In questo caso i test sono discordanti: la serie potrebbe essere stazionaria attorno a un trend deterministico. In questo caso, l'ADF potrebbe rilevare correttamente che non c'è una radice unitaria (quindi rifiuta H0), mentre il KPSS potrebbe rilevare che la serie non è strettamente stazionaria attorno a una media costante (quindi rifiuta H0). Questo fenomeno può capitare con dati con forte stagionalità e/o ciclicità. Dai plot nella fase di visualizzazione e dai correlogrammi abbiamo capito che questa è una serie particolare; intesa stazionaria (o trend-stazionaria) dal test ADF, nel senso che non esiste radice unitaria, ma il KPSS rileva effettivamente che la varianza del random walk fittato sulla serie (H0) è diversa da 0. Nel test KPSS la serie è espressa come somma di trend deterministici, random walk, ed errori stazionari. Il test è il 'Lagrange multiplier test' dell'ipotesi nulla: il random walk ha varianza 0. \
Per questi due test spesso si parla di trend-stazionarietà, nel senso che mirano a rilevare la presenza (o assenza) di un trend.

In [ ]:
# 4) DECOMPOSIZIONE DELLA SERIE: troppo lenta 30min+
# -----------------------------------------------------

In [ ]:
# 5) AUTOCORRELAZIONI per Leclerc
# -----------------------------------------------------
print("\n\n4. ANALISI DELLE CORRELAZIONI")
print("-" * 50)

# ACF - Autocorrelation Function
plot_acf(f1[f1["Driver"]=="LEC"]["RPM"].dropna(), lags=2000, alpha=0.05, title='Funzione di Autocorrelazione (ACF)')
plt.tight_layout()

# PACF - Partial Autocorrelation Function
plot_pacf(f1[f1["Driver"]=="LEC"]["RPM"], lags=20, alpha=0.05, title='Funzione di Autocorrelazione Parziale (PACF)')
plt.tight_layout()
plt.show()

La PACF va a 0 dopo 2 lag, mentre l'ACF ha alte correlazioni anche a lag molto elevati. \
Alcuni picchi si notano in prossimità dei lag 660/680 e 1320/1340, cioè quel punto della pista nei giri prima

#### ANALISI DELLE FEATURE

In [ ]:
# 6) ANALISI CON COVARIATE
# -----------------------------------------------------
# 6.1) DISTRIBUZIONE DELLE VARIABILI
# -----------------------------------------------------
print("\n\n6.1 DISTRIBUZIONE DELLE VARIABILI")
print("-" * 50)
f1lec = f1[f1["Driver"]=="LEC"]
# Seleziona solo le variabili numeriche (escluso giorno_settimana e ora_del_giorno che sono categoriche)
numeric_vars = ['DistanceToDriverAhead', 'RPM', 'Speed', 'nGear', 'Throttle','LapNumber']
vartoplot = ['DistanceToDriverAhead', 'RPM', 'Speed']

# Plot delle distribuzioni - solo istogrammi, senza boxplot
fig, axes = plt.subplots(len(vartoplot), 1, figsize=(16, 3*len(vartoplot)))
fig.suptitle('Distribuzione delle Variabili', fontsize=16, y=0.99)

for i, var in enumerate(vartoplot):
    # Histogram
    sns.histplot(f1lec[var], kde=True, ax=axes[i])
    axes[i].set_title(f'Distribuzione di {var}')
    axes[i].set_xlabel(var)
    axes[i].set_ylabel('Frequenza')

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

# -----------------------------------------------------
# 6.2) MATRICE DI CORRELAZIONE
# -----------------------------------------------------
print("\n\n6.2 MATRICE DI CORRELAZIONE")
print("-" * 50)

# Calcoliamo la matrice di correlazione per le variabili numeriche
corr_matrix = f1lec[numeric_vars].corr()

# Visualizziamo la matrice di correlazione con annotazioni
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(corr_matrix, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0,
            annot=True, fmt=".2f", square=True, linewidths=.5)

plt.title('Matrice di Correlazione', fontsize=16)
plt.tight_layout()
plt.show()

print("Correlazioni con RPM:")
for var in numeric_vars:
    if var != 'RPM':
        corr, p_value = pearsonr(f1lec['RPM'], f1lec[var])
        print(f"- {var}: {corr:.4f} (p-value: {p_value:.4g})")
# -----------------------------------------------------
# 6.3) RELAZIONI TRA VARIABILI 
# -----------------------------------------------------
print("\n\n6.3 RELAZIONI TRA VARIABILI")
print("-" * 50)
# Pairplot per le variabili numeriche
sns.pairplot(f1lec[vartoplot], height=1.5, corner=True)
plt.suptitle('Pairplot', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

# -----------------------------------------------------
# 6.4) RELAZIONI CON IL TARGET (DC_POWER)
# -----------------------------------------------------
print("\n\n6.4 RELAZIONI CON IL TARGET (DC_POWER)")
print("-" * 50)

# Crea i scatter plot con linea di regressione
fig, axes = plt.subplots(2, 3, figsize=(16, 12))
axes = axes.flatten()

for i, var in enumerate(['DistanceToDriverAhead', 'Speed', 'nGear', 'Throttle','LapNumber']):
    sns.regplot(x=f1lec[var], y=f1lec['RPM'], ax=axes[i], scatter_kws={'alpha':0.3}, line_kws={'color':'red'})
    axes[i].set_title(f'RPM vs {var}')
    axes[i].set_xlabel(var)
    axes[i].set_ylabel('RPM')

plt.tight_layout()
plt.show()
        
# -----------------------------------------------------
# 6.5) HEATMAP
# -----------------------------------------------------
print("\n\n6.6 HEATMAP PER BRAKE E NGEAR")
print("-" * 50)

# Creazione di una pivot table per la heatmap
pivot_data = f1lec.pivot_table(index='Brake', 
                            columns='nGear', 
                            values='RPM', 
                            aggfunc='mean')

plt.figure(figsize=(12, 10))
sns.heatmap(pivot_data, cmap='YlOrRd', annot=True, fmt=".1f", linewidths=.5)
plt.title('RPM')
plt.xlabel('nGear')
plt.ylabel('Brake')
plt.tight_layout()
plt.show()

In [ ]:
# 7) CROSS-CORRELATION
f1lec = f1[f1["Driver"]=="LEC"]
s1= (f1lec['RPM'] - f1lec['RPM'].mean()) / f1lec['RPM'].std()
s2 = (f1lec['Speed'] - f1lec['Speed'].mean()) / f1lec['Speed'].std()

cross_corr_norm = np.correlate(s1, s2, mode='full')
lags = np.arange(-len(s1)+1, len(s2))

plt.figure(figsize=(10, 4))
plt.plot(lags, cross_corr_norm)
plt.axhline(0, color='gray', linestyle='--')
plt.title("Cross-correlation between RPM and Speed")
plt.xlabel("Lag")
plt.ylabel("Correlation")
plt.grid(True)
plt.tight_layout()
plt.show()

Sembra il caso di due serie che covariano all'unisono. D'altronde ha senso pensare che la velocità e le RPM covarino insieme nel tempo, senza che una anticipi l'altra.

### <a id="electricity-demand"></a>ELECTRICITY DEMAND

Total electricity demand in GW for Victoria state, Australia, every half-hour during 2014. \
Per 'Electricity demand' si intende la quantità totale di energia elettrica richiesta da un certo sistema o area geografica in un dato momento. È una misura fondamentale nel settore dell’energia perché consente di capire quanta energia serve per soddisfare i bisogni di consumatori, aziende, infrastrutture. Questo ammontare esclude l’energia degli impianti solari sui tetti, dei parchi eolici molto piccoli e trigenerazione di gas.


In [ ]:
#0) uploading data
elec = loaded_data['elec']
def add_midnight_time(date_str):
    if len(date_str) < 19:  # Se la stringa è più corta di "2021-01-01 00:00:00"
        return date_str + " 00:00:00"
    return date_str

elec['Date'] = elec['Date'].apply(add_midnight_time)
elec['Date'] = pd.to_datetime(elec['Date'])
elec.set_index('Date',inplace=True)

Per le rilevazioni a mezzanotte (00:00:00) l'indice di riga dava solo la forma YYYY-MM-GG, quindi ho aggiustato per rendere possibili tutte le analisi e avere gli indici tutti nello stesso formato.

In [ ]:
# 1) DESCRIZIONE SERIE STORICA
# -----------------------------------------------------
print("1. DESCRIZIONE DELLA SERIE STORICA")
print("-" * 50)

#### Informazioni generali sulla serie
print(f"Dimensione della serie: {elec.shape[0]}")
#print(f"Frequenza: {getattr(oil.index, 'freq', 'Non specificata')}")
print("Frequenza ogni 30 min -->>",elec.index[1:3])
print("\nStatistiche descrittive:")
print(elec.describe())
print("\nInformazioni aggiuntive sulla serie:")
print(elec.info())

Abbiamo anche la temperatura esterna e la dummy Workday che si accende (diventa 1) se si è in un giorno lavorativo

In [ ]:
# 2) VISUALIZZAZIONE SERIE STORICA
# -----------------------------------------------------
series_dict = {'Victoria (Australia) Electricity Demand': elec["Demand"]}
ts_plotly(series_dict, title="Victoria (Australia) Electricity Demand")  

In Australia l'inverno va dal 21 giugno al 21 settembre.

In [ ]:
# 3) ANALISI STAZIONARIETÀ
# -----------------------------------------------------
print("\n\n3. ANALISI STAZIONARIETÀ")
print("-" * 50)
test_adf(elec["Demand"])
test_kpss(elec["Demand"])

In questo caso i test sono discordanti: la serie potrebbe essere stazionaria attorno a un trend deterministico. In questo caso, l'ADF potrebbe rilevare correttamente che non c'è una radice unitaria (quindi rifiuta H0), mentre il KPSS potrebbe rilevare che la serie non è strettamente stazionaria attorno a una media costante (quindi rifiuta H0). Questo fenomeno può capitare con dati con forte stagionalità e/o ciclicità. Dai plot nella fase di visualizzazione e dai correlogrammi abbiamo capito che questa è una serie particolare; intesa stazionaria (o trend-stazionaria) dal test ADF, nel senso che non esiste radice unitaria, ma il KPSS rileva effettivamente che la varianza del random walk fittato sulla serie (H0) è diversa da 0. Nel test KPSS la serie è espressa come somma di trend deterministici, random walk, ed errori stazionari. Il test è il 'Lagrange multiplier test' dell'ipotesi nulla: il random walk ha varianza 0. \
Per questi due test spesso si parla di trend-stazionarietà, nel senso che mirano a rilevare la presenza (o assenza) di un trend.

In [ ]:
# 4) DECOMPOSIZIONE DELLA SERIE
# -----------------------------------------------------
print("\n\n4. DECOMPOSIZIONE DELLA SERIE")
print("-" * 50)
decomposition_add = seasonal_decompose(elec['Demand'], model='additive', period=336)
decomposition_mult = seasonal_decompose(elec['Demand'], 
                                            model='multiplicative', period=336)
plot_decomposition(decomposition_add, decomposition_mult)


In questo caso fra additiva o moltiplicativa non c'è visibile differenza

In [ ]:
# 5) AUTOCORRELAZIONI
# -----------------------------------------------------
print("\n\n4. ANALISI DELLE CORRELAZIONI")
print("-" * 50)

# ACF - Autocorrelation Function
plot_acf(elec['Demand'], lags=200, alpha=0.05, title='Funzione di Autocorrelazione (ACF)')
plt.tight_layout()

# PACF - Partial Autocorrelation Function
plot_pacf(elec['Demand'], lags=25, alpha=0.05, title='Funzione di Autocorrelazione Parziale (PACF)')
plt.tight_layout()
plt.show()

**ACF:** Mostra un chiaro pattern ciclico con picchi che si ripetono regolarmente
I picchi sono distanziati di circa 48 lag, che corrisponde a 24 ore, questo indica una forte stagionalità giornaliera nella domanda di elettricità. L'ampiezza dei picchi diminuisce lentamente nel tempo, suggerendo una persistenza del pattern giornaliero. I valori negativi tra i picchi indicano che c'è un'inversione della correlazione a metà del ciclo giornaliero

**PACF:** Ha un valore positivo molto alto al lag 1, e uno fortemente negativo al lag 2. Va praticamente a 0 dopo il lag 3. Interessante notare al lag 16 (8 ore) vi è una correlazione di circa +0.25\
La domanda di elettricità in Australia segue un chiaro pattern giornaliero (ogni 24 ore). C'è una forte dipendenza dal valore precedente (mezz'ora prima).
Il pattern è probabilmente legato ai comportamenti quotidiani: maggiore domanda durante il giorno e minore durante la notte. Sono pattern tipici dei dati di consumo energetico, che seguono i ritmi delle attività umane e commerciali durante la giornata.

#### ANALISI DELLE FEATURES

In [ ]:
#adding features
df_reset = elec.reset_index()
df_reset.rename(columns={'Date':'DATE_TIME'}, inplace=True)
df_reset['DATE_TIME'] = pd.to_datetime(df_reset['DATE_TIME'])
df_reset['giorno_settimana'] = df_reset['DATE_TIME'].dt.dayofweek + 1
df_reset['ora_del_giorno'] = df_reset['DATE_TIME'].dt.hour + 1
df_reset['settimana_del_mese'] = ((df_reset['DATE_TIME'].dt.day - 1) // 7) + 1
df_reset['settimana_del_anno'] = df_reset['DATE_TIME'].dt.isocalendar().week
df_reset['giorno_del_mese'] = df_reset['DATE_TIME'].dt.day
df_reset['mese_del_anno'] = df_reset['DATE_TIME'].dt.month
df_elec = df_reset.set_index('DATE_TIME')

In [ ]:
#boxplots
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
# Boxplot per Category1
df_elec.boxplot(column='Demand', by='giorno_settimana', ax=axes[0])
axes[0].set_xlabel('Giorno della settimana')
axes[0].set_ylabel('Domanda di elettricità (GW)')

# Boxplot per Category2
df_elec.boxplot(column='Demand', by='ora_del_giorno', ax=axes[1])
axes[1].set_xlabel('Ora del giorno')
axes[1].set_ylabel('Domanda di elettricità (GW)')

# Boxplot per Category3
df_elec.boxplot(column='Demand', by='mese_del_anno', ax=axes[2])
axes[2].set_xlabel("Mese dell'anno")
axes[2].set_ylabel('Domanda di elettricità (GW)')

# Aggiungi spazio tra i subplot
plt.tight_layout()
plt.suptitle('')  
plt.show()

Da questi grafici capiamo chiaramente la dipendenza temporale della domanda di elettricità. Nel weekend e nella viene richiesta meno (come era lecito aspettarsi). I picchi durante la giornata sono dalle 19 alle 21 (cucine e rientri a casa?);Dal lunedì al venerdì non sembra esserci sostanziale differenza. Inoltre notiamo come il picco l'abbiamo nel mese di Luglio (il più freddo in Australia); successivamente troviamo Gennaio e Febbraio (condizionatori per il troppo caldo?).

In [ ]:
# 6) ANALISI CON COVARIATE
# -----------------------------------------------------
# 6.1) DISTRIBUZIONE DELLE VARIABILI
# -----------------------------------------------------
print("\n\n6.1 DISTRIBUZIONE DELLE VARIABILI")
print("-" * 50)
print("Demand in GW and Temperature in C°")

numeric_vars = df_elec.columns
fig, axes = plt.subplots(2, 1, figsize=(16, 6))
fig.suptitle('Distribuzione delle Variabili', fontsize=16, y=0.99)

for i, var in enumerate(df_elec[['Demand','Temperature']]):
    sns.histplot(df_elec[var], kde=True, ax=axes[i])
    axes[i].set_title(f'Distribuzione di {var}')
    axes[i].set_xlabel(var)
    axes[i].set_ylabel('Frequenza')

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

# -----------------------------------------------------
# 6.2) MATRICE DI CORRELAZIONE
# -----------------------------------------------------
print("\n\n6.2 MATRICE DI CORRELAZIONE")
print("-" * 50)

corr_matrix = df_elec[numeric_vars].corr()

plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(corr_matrix, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0,
            annot=True, fmt=".2f", square=True, linewidths=.5)

plt.title('Matrice di Correlazione', fontsize=16)
plt.tight_layout()
plt.show()

print("Correlazioni con Demand:")
for var in numeric_vars:
    if var != 'Demand':
        corr, p_value = pearsonr(df_elec['Demand'], df_elec[var])
        print(f"- {var}: {corr:.4f} (p-value: {p_value:.4g})")
# -----------------------------------------------------
# 6.3) RELAZIONI TRA VARIABILI 
# -----------------------------------------------------
print("\n\n6.3 RELAZIONI TRA VARIABILI")
print("-" * 50)
pairplotvars = ['Demand', 'Workday', 'Temperature', 'giorno_settimana',
       'ora_del_giorno', 'mese_del_anno']
# Pairplot per le variabili numeriche
sns.pairplot(df_elec[pairplotvars], height=2.5, corner=True)
plt.suptitle('Pairplot', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

# -----------------------------------------------------
# 6.4) RELAZIONI CON IL TARGET (DC_POWER)
# -----------------------------------------------------
print("\n\n6.4 RELAZIONI CON IL TARGET (Demand)")
print("-" * 50)

# Definisci le variabili predittori
predictors = ['Workday', 'Temperature', 'giorno_settimana',
       'ora_del_giorno', 'settimana_del_mese', 'settimana_del_anno',
       'giorno_del_mese', 'mese_del_anno']

# Crea i scatter plot con linea di regressione
fig, axes = plt.subplots(2, 4, figsize=(16, 12))
axes = axes.flatten()

for i, var in enumerate(predictors):
    sns.regplot(x=df_elec[var], y=df_elec['Demand'], ax=axes[i], scatter_kws={'alpha':0.3}, line_kws={'color':'red'})
    axes[i].set_title(f'Demand vs {var}')
    axes[i].set_xlabel(var)
    axes[i].set_ylabel('Demand')

plt.tight_layout()
plt.show()

# -----------------------------------------------------
# 6.5) ANALISI TEMPORALE
# -----------------------------------------------------
print("\n\n6.5 DEMAND VS TEMPERATURE")
print("-" * 50)

fig, ax1 = plt.subplots(figsize=(16, 8))
        
ax1.set_xlabel('Data')
ax1.set_ylabel('Demand', color='tab:blue')
ax1.plot(df_elec.index, df_elec['Demand'], color='tab:blue', label='Demand')
ax1.tick_params(axis='y', labelcolor='tab:blue')
        
ax2 = ax1.twinx()  # crea un secondo asse y condividendo l'asse x
ax2.set_ylabel('Temperature', color='tab:orange')  
ax2.plot(df_elec.index, df_elec['Temperature'], color='tab:orange', label='Temperature')
ax2.tick_params(axis='y', labelcolor='tab:orange')
        
fig.tight_layout()
plt.title('Demand VS Temperature nel Tempo')
        
# Aggiungi legenda
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')
plt.show()
        
# -----------------------------------------------------
# 6.6) HEATMAP PER ORA E GIORNO
# -----------------------------------------------------
print("\n\n6.6 HEATMAP PER ORA E GIORNO")
print("-" * 50)

# Creazione di una pivot table per la heatmap
pivot_data = df_elec.pivot_table(index='ora_del_giorno', 
                            columns='giorno_settimana', 
                            values='Demand', 
                            aggfunc='mean')

plt.figure(figsize=(12, 10))
sns.heatmap(pivot_data, cmap='YlOrRd', annot=True, fmt=".1f", linewidths=.5)
plt.title('Domanda di elettricità media per Ora e Giorno della settimana')
plt.xlabel('Giorno della Settimana')
plt.ylabel('Ora del Giorno')
plt.tight_layout()
plt.show()

# 6.7) HEATMAP 2
# -----------------------------------------------------
print("\n\n6.6 HEATMAP PER MESE E ORA DEL GIORNO")
print("-" * 50)

# Creazione di una pivot table per la heatmap
pivot_data = df_elec.pivot_table(index='ora_del_giorno', 
                            columns='mese_del_anno', 
                            values='Demand', 
                            aggfunc='mean')

plt.figure(figsize=(12, 10))
sns.heatmap(pivot_data, cmap='YlOrRd', annot=True, fmt=".1f", linewidths=.5)
plt.title('Domanda di elettricità media per mese e ora del giorno')
plt.xlabel('mese')
plt.ylabel('ora del giorno')
plt.tight_layout()
plt.show()

# ===========================
# 6.8) Seasonal plot
# ===========================
# Mapping month numbers to names
month_names = {
1: 'January', 2: 'February', 3: 'March', 4: 'April',
5: 'May', 6: 'June', 7: 'July', 8: 'August',
9: 'September', 10: 'October', 11: 'November', 12: 'December'
}

plt.figure(figsize=(12, 8))
palette = sns.color_palette("viridis", n_colors=len(df_elec['mese_del_anno'].unique()))
for i, month in enumerate(df_elec['mese_del_anno'].unique()):
    monthly_data = df_elec[df_elec['mese_del_anno'] == month]
    daily_demand = monthly_data.groupby('giorno_settimana')['Demand'].mean()
    plt.plot(daily_demand, label=f'{month_names[month]}', color=palette[i])

plt.title('Seasonal Electricity Demand')
plt.xlabel('Day of the week')
plt.ylabel('Average Demand (GW)')
plt.legend(title='Month')
plt.grid(True)
plt.show()



Questi grafici aiutano a farci capire meglio il fenomeno in questione e danno una prospettiva diversa rispetto a ciò di cui abbiamo parlato anche sopra.

In [ ]:
# 7) CROSS-CORRELATION
demand_norm = (df_elec['Demand'] - df_elec['Demand'].mean()) / df_elec['Demand'].std()
temp_norm = (df_elec['Temperature'] - df_elec['Temperature'].mean()) / df_elec['Temperature'].std()

cross_corr_norm = np.correlate(demand_norm, temp_norm, mode='full')
lags = np.arange(-len(temp_norm)+1, len(demand_norm))

plt.figure(figsize=(10, 4))
plt.plot(lags, cross_corr_norm)
plt.axhline(0, color='gray', linestyle='--')
plt.title("Cross-correlation between Demand and Temperature")
plt.xlabel("Lag")
plt.ylabel("Correlation")
plt.grid(True)
plt.tight_layout()
plt.show()

In generale dobbiamo dire che la struttura di correlazione è complessa e non lineare nel corso dell'anno.
Si osserva un chiaro pattern oscillatorio che suggerisce una relazione ciclica tra temperatura e domanda di elettricità.
Intorno ai primi lag negativi sembra esserci un picco di correlazione positiva. Dimostrerebbe che la temperatura anticipa/prevede la domanda di elettricità.\
Il grafico mostra picchi positivi a lag positivi (7500 circa) e picchi positivi a lag negativi intorno a -6000 lag. Notiamo inoltre picchi negativi a lag positivi (12000/13000 circa) e picchi negativi a lag negativi intorno a -9000/-10000 lag.\

In inverno, temperature più basse portano a maggiore domanda per il riscaldamento. In estate c'è meno bisogno, ma prima abbiamo visto che se è molto caldo, le temperature particolarmente alte possono portare a maggiore domanda per il condizionamento.\

La complessa struttura della cross-correlation evidenzia come la relazione tra temperatura e domanda elettrica vari significativamente nel corso di diversi periodi temporali.

### <a id="favorita-sales"></a>FAVORITA SALES (lo stesso di prima)

Precedentemente, per questo dataset, avevamo trattato fino al punto 5 (senza variabili aggiuntive). Ora estendiamo l'analisi.

In [ ]:
# 0) uploading data
favorita_train = loaded_data['favorita_train']
#favorita_train.set_index("date", inplace=True)
favorita_train.drop(columns=['id'],inplace=True)
favorita_train.rename(columns={"store_nbr": "Store","family":"Product_type"}, inplace=True)

In [ ]:
# DESCRIZIONE SERIE STORICHE
# -----------------------------------------------------
print("1. DESCRIZIONE DELLA SERIE STORICHE")
print("-" * 50)

#### Informazioni generali sulla serie
print(f"Dimensione della serie: {favorita_train.shape[0]}")
print("\nStatistiche descrittive:")
print(favorita_train.describe(include="all").round(2))
print("\nInformazioni aggiuntive sulla serie:")
print(favorita_train.info())

Ci sono 3 milioni di record. Vendite dei supermercati Favorita in Ecuador. Abbiamo l'identificativo di store, il tipo di prodotto, le vendite totali per un prodotto in uno store in un giorno, e la variabile 'onpromotion' che dice quanti prodotti, di un determinato tipo di prodotto, erano in promozione in quel giorno e in quello store.

In [ ]:
# Altre informazione del dataset
print("Quanti stores ci sono?",favorita_train["Store"].unique().shape[0]) #sono 54 stores
print("Quanti tipi di prodotto diverso ci sono?",favorita_train["Product_type"].unique().shape[0]) #ci sono 33 tipi di prodotto
print("Quanti giorni abbiamo?",favorita_train.index.unique().shape[0]) #ci sono 1684 giorni
print("Top 5 store in termini di sales totali",
      favorita_train[["Store", "sales"]].groupby(["Store"]).mean().sort_values(by="sales", ascending=False).head())
print("Top 5 tipi di prodotto in termini di sales totali",
      favorita_train[["Product_type", "sales"]].groupby(["Product_type"]).mean().sort_values(by="sales",
                                                                                             ascending=False).head())

In [ ]:
#Adding features
#Creo una dummy per le promozione
favorita_train['promo'] = favorita_train['onpromotion'].apply(lambda x: 0 if x == 0 else 1)
#aggiungo il prezzo del petrolio come regressore, perchè l'Ecuador è un paese molto dipendente dal petrolio
oil = loaded_data['daily_oil']
oil.fillna(0,inplace=True)
favorita_train = pd.merge(favorita_train, oil, on='date',how='left')
favorita_train.rename(columns={"dcoilwtico":"oil price"},inplace=True)
#adding temporal features
df_reset = favorita_train.reset_index()
df_reset.rename(columns={'date':'DATE_TIME'}, inplace=True)
df_reset['DATE_TIME'] = pd.to_datetime(df_reset['DATE_TIME'])

df_reset['giorno_settimana'] = df_reset['DATE_TIME'].dt.dayofweek + 1
df_reset['settimana_del_mese'] = ((df_reset['DATE_TIME'].dt.day - 1) // 7) + 1
df_reset['settimana_del_anno'] = df_reset['DATE_TIME'].dt.isocalendar().week
df_reset['giorno_del_mese'] = df_reset['DATE_TIME'].dt.day
df_reset['mese_del_anno'] = df_reset['DATE_TIME'].dt.month
favorita_train = df_reset.set_index('DATE_TIME')


In [ ]:
#boxplot sales by promotion
plt.figure(figsize=(10,6))
sns.boxplot(x=favorita_train["promo"], 
            y=np.log1p(favorita_train["sales"]))  # Log transform for better visibility
plt.title("Promotion Impact on Sales (Log Scale)")
plt.xlabel("Item on Promotion")
plt.ylabel("Log(Sales + 1)")
plt.xticks([0,1], ["No Promotion", "Promoted"])
plt.show()

Notiamo come le vendite di quel prodotto aumentano se si mette in promozione (prevedibile).

In [ ]:
#boxplot sales by day of week
dow_sales = favorita_train.groupby("giorno_settimana")["sales"].mean().reset_index()
sns.barplot(x="giorno_settimana", y="sales", data=dow_sales)
plt.title("Average Sales by Day of Week")
plt.xticks(rotation=45)
plt.show()

Il weekend si compra di più.

In [ ]:
#Controlliamo valori mancanti
print("VALORI MANCANTI:",favorita_train.isna().sum())

In [ ]:
# Quando abbiamo i valori mancanti?
nan_days = favorita_train[favorita_train['oil price'].isna()]['giorno_settimana'].value_counts()
print(nan_days) 

I giorni in cui abbiamo valori mancanti nel prezzo del petrolio sono sabato e domenica, nel quale i mercati sono chiusi

Prendiamo le vendite dello store 1, lo stesso di prima e analizziamo le vendite rispetto ad alcune variabili

In [ ]:
#ricreo i dati per un singolo store, ma con le variabili 
store1 = favorita_train.groupby('DATE_TIME').agg({
    'sales':'sum',
    'Store': 'first',
    'Product_type': 'first',
    'promo': 'first',
    'oil price': 'first',
    'giorno_settimana': 'first',
    'settimana_del_mese': 'first',
    'settimana_del_anno': 'first',
    'giorno_del_mese': 'first',
    'mese_del_anno': 'first'
}).reset_index()
store1.set_index("DATE_TIME", inplace=True)
store1['settimana_del_anno'] = store1['settimana_del_anno'].astype(int)
#store1["Product_type"] = store1["Product_type"].astype("category")
store1.drop(columns=['Product_type'],inplace=True)


In [ ]:
## 6) ANALISI CON COVARIATE
# ===========================
# 1. Pairplot 
# ===========================
sns.pairplot(store1[['sales','oil price']], height=2.5, corner=True)
plt.suptitle("Pairplot delle variabili numeriche", y=1.02)
plt.show()

# ===========================
# 2. Relazione col target 'sales'
# ===========================
predictors = ['oil price','promo','giorno_settimana']
fig, axes = plt.subplots(len(predictors), 1, figsize=(10, 5 * len(predictors)))

for i, var in enumerate(predictors):
    sns.regplot(x=store1[var], y=store1['sales'], ax=axes[i], 
                scatter_kws={'alpha': 0.3}, line_kws={'color': 'red'})
    axes[i].set_title(f'Relazione tra {var} e Sales')

plt.tight_layout()
plt.show()

# ===========================
# 3. Sovrapposizione serie sales e oil price
# ===========================
fig, ax1 = plt.subplots(figsize=(14, 6))

ax1.set_title("Store 1 sales vs Oil Price")
ax1.set_xlabel("Data")
ax1.set_ylabel("Sales", color="blue")
ax1.plot(store1.index, store1['sales'], color="blue", label='Sales')

ax2 = ax1.twinx()
ax2.set_ylabel("Oil Price", color="green")
ax2.plot(store1.index, store1['oil price'], color="green", label='Oil Price')

fig.tight_layout()
plt.show()

# ===========================
# 4. Heatmap: sales per giorno della settimana e mese dell'anno
# ===========================
pivot_data = store1.pivot_table(index='mese_del_anno', 
                            columns='giorno_settimana', 
                            values='sales', 
                            aggfunc='mean')

plt.figure(figsize=(12, 8))
sns.heatmap(pivot_data, cmap='YlOrRd', annot=True, fmt=".1f", linewidths=0.5)
plt.title("Sales medie per giorno della settimana e mese dell'anno")
plt.ylabel("Mese dell'anno")
plt.xlabel("Giorno della settimana")
plt.show()

# ===========================
# 5. Matrice di correlazione
# ===========================
corr = store1.drop(columns=['Store']).corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Matrice di correlazione")
plt.show()


# ===========================
# 6. Seasonal plot
# ===========================
store1['year'] = store1.index.year
store1['month'] = store1.index.month
store1['year'] = store1.index.year
plt.figure(figsize=(12, 6))
palette = sns.color_palette("viridis", n_colors=len(store1['year'].unique()))
for i, year in enumerate(store1['year'].unique()):
    yearly_data = store1[store1['year'] == year]
    monthly_sales = yearly_data.groupby('month')['sales'].mean()
    plt.plot(monthly_sales, label=f'{year}', color=palette[i])

plt.title('Seasonal Sales Plot')
plt.xlabel('Month')
plt.ylabel('Average Sales')
plt.legend(title='Year',loc="lower left",fontsize=7)
plt.grid(True)
plt.show()

Interessante vedere che Dicembre è per distacco il mese con più vendite, ci sono tutte le festività natalizie e di capodanno; ha senso (91% della popolazione ecuadorena è cristiana cattolica). Dall'ultimo seasonal plot vediamo come in generale le vendite di Favorita sono andate in aumento.

In [ ]:
# 7) CROSS-CORRELATION
s1_norm = (store1['sales'] - store1['sales'].mean()) / store1['sales'].std()
store1['oilprice_inter'] = store1['oil price'].fillna(method='ffill').fillna(method='bfill')
s2_norm = (store1['oilprice_inter'] - store1['oilprice_inter'].mean()) / store1['oilprice_inter'].std()

cross_corr_norm = np.correlate(s1_norm, s2_norm, mode='full')
lags = np.arange(-len(s1_norm)+1, len(s2_norm))

plt.figure(figsize=(10, 4))
plt.plot(lags, cross_corr_norm)
plt.axhline(0, color='gray', linestyle='--')
plt.title("Cross-correlation between Sales and Oil price")
plt.xlabel("Lag")
plt.ylabel("Correlation")
plt.grid(True)
plt.tight_layout()
plt.show()

A breve termine, le vendite reagiscono negativamente al prezzo del petrolio (es. forse per l’effetto sui costi). Guarda il intorno allo 0 per capire meglio. Quando il prezzo del petrolio sale, circa 100 lag dopo le vendite diminuiscono.\
Nel lungo termine, c'è una correlazione positiva → potenzialmente perché il petrolio è anche un indicatore economico (quando l'economia gira, il prezzo sale e anche le vendite?). 